In [17]:
import os
import cv2 as cv
import numpy as np
import shutil 

Clear duplicates from annotations 

In [2]:
for i in range(6804):
    with (open(f"ann/obj_train_data/frame_{str(i).zfill(6)}.txt") as f1, 
          open(f"ann/labels/frame_{str(i).zfill(6)}.txt", "w") as f2):
        f2.writelines(set(f1.readlines()))

Load fiftyone for dataset visualisation

In [ ]:
import fiftyone as fo

# fo.utils.video.reencode_video("/vids/IMG_1855.MP4", "/vids/NEW_IMG_1855.MP4")

# Create a dataset from a list of videos
dataset = fo.Dataset.from_videos(
    ["./vids/IMG_1855.MP4"]
)
    
session = fo.launch_app(dataset)

session.wait()

Get images for dataset with big mathematical distance between them and move them

In [31]:
TRAIN_OR_VAL = "val"

In [32]:
MAX_DIFF_HYPERPARAM = 7e6
IMAGES_STEP = 25
to_skip = set()
images_path = os.getcwd() + f'\\datasets\\aero2_init\\images\\{TRAIN_OR_VAL}'
images = sorted(os.listdir(images_path))
for i in range(len(images) // IMAGES_STEP + 1):
    ref_img = images[i*IMAGES_STEP]
    similar_to_ref = set()
    left_border = i*IMAGES_STEP + 1
    right_border = min((i+1)*IMAGES_STEP, len(images))
    img1 = cv.imread(f'{images_path}\\{ref_img}', cv.IMREAD_GRAYSCALE)
    for j in range(left_border, right_border):
        img2 = cv.imread(f'{images_path}\\{images[j]}', cv.IMREAD_GRAYSCALE)
        if np.sum(cv.absdiff(img1, img2)) <= MAX_DIFF_HYPERPARAM:
            similar_to_ref.add(f'{images_path}\\{images[j]}')
    to_skip.update(similar_to_ref)

print(f'{len(to_skip)} to skip from {len(images)}')

843 to skip from 1230


Move images for train or val

In [33]:
images_path = os.getcwd() + f'\\datasets\\aero2_init\\images\\{TRAIN_OR_VAL}'
labels_path = os.getcwd() + f'\\datasets\\aero2_init\\labels\\{TRAIN_OR_VAL}'
images_new_path = os.getcwd() + f'\\datasets\\aero2\\images\\{TRAIN_OR_VAL}'
labels_new_path = os.getcwd() + f'\\datasets\\aero2\\labels\\{TRAIN_OR_VAL}'
for img_name in os.listdir(images_path):
    img_filename = f'{images_path}\\{img_name}'
    if img_filename not in to_skip:
        shutil.copy(img_filename, f"{images_new_path}\\{img_name}")
        label_name = f"{img_name.split('.')[0]}.txt"
        shutil.copy(
            f"{labels_path}\\{label_name}",
            f"{labels_new_path}\\{label_name}")


Cut frames from video

Get video from frames

In [ ]:
for path in imgs_paths:
    fourcc = cv.VideoWriter_fourcc(*'mp4v')
    out_name = '_'.join(path.split('\\')) + ".mp4"
    out = cv.VideoWriter(out_name, fourcc, 30.0, resol[path.split('\\')[1]])
    images = [f"{os.getcwd()}{path}{i}" for i in sorted(os.listdir(os.getcwd() + path))]
    for img in images:
            img_cv = cv.cvtColor(cv.imread(img), cv.COLOR_BGR2RGB)
            out.write(img_cv)
    out.release()